# Clean folders

In [1]:
import os
import shutil
import subprocess
import sys
import threading
import time
from datetime import datetime as dt
from glob import glob

MAIN_PATH = "speed_test/"
TEMP_PATH = f"{MAIN_PATH}temp/"
RAW_PATH = f"{MAIN_PATH}raw_data/"
CHECK_POINT = f"{MAIN_PATH}_checkpoint"
os.makedirs(RAW_PATH, exist_ok=True)


def clean_project():
    try:
        shutil.rmtree("spark-warehouse")
        shutil.rmtree("metastore_db")
        shutil.rmtree("speed_test/_checkpoint")
    except:
        pass


clean_project()

# SPEED TEST

In [2]:
def speed_test():

    def move_temp_to_raw():
        files = glob(f"{TEMP_PATH}*.json")
        list(map(lambda x: os.rename(x, x.replace(TEMP_PATH, RAW_PATH)), files))

    now = dt.now()
    file_name = f"log_{now.strftime('%Y_%m_%d_%H_%M_%s')}.json"
    subprocess.run(
        f"touch {TEMP_PATH}{file_name} && speedtest --format=json >> {TEMP_PATH}{file_name} && mv  {TEMP_PATH}{file_name} {RAW_PATH}{file_name}",
        shell=True,
        executable="/bin/bash",
    )
    move_temp_to_raw()
    list(
        map(
            os.remove,
            filter(
                lambda x: os.path.getsize(x) == 0, glob("speed_test/raw_data/*.json")
            ),
        )
        )
    return dt

# DeltaLake

In [3]:
import pyspark.sql.functions as F
from delta import *
from pyspark.sql.types import *

sql = lambda statement, limit=5: spark.sql(statement).limit(limit).toPandas()
minutes_to_seconds = lambda x: x * 60
run_every_n_seconds = minutes_to_seconds(10)

## Introduction
Delta Lake is an open source project that enables building a Lakehouse architecture on top of data lakes. Delta Lake provides ACID transactions, scalable metadata handling, and unifies streaming and batch data processing on top of existing data lakes, such as S3, ADLS, GCS, and HDFS.

Specifically, Delta Lake offers:

- ACID transactions on Spark: Serializable isolation levels ensure that readers never see inconsistent data.

- Scalable metadata handling: Leverages Spark distributed processing power to handle all the metadata for petabyte-scale tables with billions of files at ease.

- Streaming and batch unification: A table in Delta Lake is a batch table as well as a streaming source and sink. Streaming data ingest, batch historic backfill, interactive queries all just work out of the box.

- Schema enforcement: Automatically handles schema variations to prevent insertion of bad records during ingestion.

- Time travel: Data versioning enables rollbacks, full historical audit trails, and reproducible machine learning experiments.

- Upserts and deletes: Supports merge, update and delete operations to enable complex use cases like change-data-capture, slowly-changing-dimension (SCD) operations, streaming upserts, and so on.

## What Is a Lakehouse?
Few systems are beginning to emerge that address the limitations of data lakes. A lakehouse is a new, open architecture that combines the best elements of data lakes and data warehouses. Lakehouses are enabled by a new system design: implementing similar data structures and data management features to those in a data warehouse directly on top of low cost cloud storage in open formats. They are what you would get if you had to redesign data warehouses in the modern world, now that cheap and highly reliable storage (in the form of object stores) are available.

A lakehouse has the following key features:

- Transaction support: In an enterprise lakehouse many data pipelines will often be reading and writing data concurrently. Support for ACID transactions ensures consistency as multiple parties concurrently read or write data, typically using SQL.
Schema enforcement and governance: The Lakehouse should have a way to support schema enforcement and evolution, supporting DW schema architectures such as star/snowflake-schemas. The system should be able to reason about data integrity, and it should have robust governance and auditing mechanisms.
- BI support: Lakehouses enable using BI tools directly on the source data. This reduces staleness and improves recency, reduces latency, and lowers the cost of having to operationalize two copies of the data in both a data lake and a warehouse.
Storage is decoupled from compute: In practice this means storage and compute use separate clusters, thus these systems are able to scale to many more concurrent users and larger data sizes. Some modern data warehouses also have this property.
- Openness: The storage formats they use are open and standardized, such as Parquet, and they provide an API so a variety of tools and engines, including machine learning and Python/R libraries, can efficiently access the data directly.
- Support for diverse data types ranging from unstructured to structured data: The lakehouse can be used to store, refine, analyze, and access data types needed for many new data applications, including images, video, audio, semi-structured data, and text.
- Support for diverse workloads: including data science, machine learning, and SQL and analytics. Multiple tools might be needed to support all these workloads but they all rely on the same data repository.
- End-to-end streaming: Real-time reports are the norm in many enterprises. Support for streaming eliminates the need for separate systems dedicated to serving real-time data applications.

<center> <img src='img/warehouseVSlakeVslakehouse.png'> </center>

## Medallion Arquitecture (Multi Hopp):

What is a medallion architecture?
A medallion architecture is a data design pattern used to logically organize data in a lakehouse, with the goal of incrementally and progressively improving the structure and quality of data as it flows through each layer of the architecture (from Bronze ⇒ Silver ⇒ Gold layer tables). Medallion architectures are sometimes also referred to as "multi-hop" architectures.

<center> <img src='img/multihop.png'> </center>


### Bronze layer (raw data)
The Bronze layer is where we land all the data from external source systems. The table structures in this layer correspond to the source system table structures "as-is," along with any additional metadata columns that capture the load date/time, process ID, etc. The focus in this layer is quick Change Data Capture and the ability to provide an historical archive of source (cold storage), data lineage, auditability, reprocessing if needed without rereading the data from the source system.

In [4]:
SCHEMA = "speed_test"

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA} ")

spark.sql(
    "set spark.databricks.delta.changeDataFeed.timestampOutOfRange.enabled = true;"
)
spark.sql("set SQLConf.ADAPTIVE_EXECUTION_ENABLED.key= true")

BRONZE_TABLE = f"{SCHEMA}.speed_test_logs"

spark.sql(
    f"""
  CREATE  TABLE IF NOT EXISTS {BRONZE_TABLE} 
  (`timestamp` Timestamp
  ) USING delta
  TBLPROPERTIES (delta.enableChangeDataFeed = true)
"""
)

SILVER_TABLE = f'{SCHEMA}.silver_speed_test_logs'
spark.sql(
    f"""
  CREATE  TABLE IF NOT EXISTS {SILVER_TABLE } 
  (`timestamp` Timestamp
  ) USING delta
  TBLPROPERTIES (delta.enableChangeDataFeed = true)
"""
)

24/04/15 21:06:17 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/15 21:06:17 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/15 21:06:20 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/15 21:06:20 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore hadoop@127.0.1.1
24/04/15 21:06:20 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/04/15 21:06:20 WARN ObjectStore: Failed to get database speed_test, returning NoSuchObjectException
24/04/15 21:06:20 WARN ObjectStore: Failed to get database speed_test, returning NoSuchObjectException
24/04/15 21:06:20 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/04/15 21:06:20 WARN ObjectStore: Failed to get database speed_test, returnin

DataFrame[]

In [5]:
sql(f"DESCRIBE HISTORY  {BRONZE_TABLE}")

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,0,2024-04-15 21:06:21.458,None,None,CREATE TABLE,"{'description': None, 'partitionBy': '[]', 'pr...",None,None,None,NaN,Serializable,True,{},None,Apache-Spark/3.5.1 Delta-Lake/3.1.0


In [6]:
sql(f"SELECT * FROM {BRONZE_TABLE}")

,timestamp


In [10]:
def update_bronze_table():

    streaming_logs_schema = StructType(
        [
            StructField(
                "download",
                StructType(
                    [
                        StructField("bandwidth", LongType(), True),
                        StructField("bytes", LongType(), True),
                        StructField("elapse", LongType(), True),
                        StructField("latency", StringType(), True),
                    ]
                ),
                True,
            ),
            StructField(
                "ping",
                StructType(
                    [
                        StructField("jitter", FloatType(), True),
                        StructField("latency", FloatType(), True),
                        StructField("low", FloatType(), True),
                        StructField("high", FloatType(), True),
                    ]
                ),
                True,
            ),
            StructField("isp", StringType(), True),
            StructField(
                "result",
                StructType(
                    [
                        StructField("id", StringType(), True),
                        StructField("url", StringType(), True),
                        StructField("persisted", BooleanType(), True),
                    ]
                ),
                True,
            ),
            StructField("server", StructType([ 
                    StructField("id", StringType(), True),
                        StructField("host", StringType(), True),
                        StructField("port", LongType(), True),
                        StructField('name',StringType(),True),
                        StructField('location',StringType(),True),
                        StructField('country',StringType(),True),
                        StructField('ip',StringType(),True)]),True),
            StructField("timestamp", TimestampType(), True),
            StructField("type", StringType(), True),
            StructField(
                "upload",
                StructType(
                    [
                        StructField("bandwidth", LongType(), True),
                        StructField("bytes", LongType(), True),
                        StructField("elapse", LongType(), True),
                        StructField("latency", StringType(), True),
                    ]
                ),
                True,
            ),
        ]
    )

    streaming_logs_sdf = (
        spark.readStream.option("ignoreCorruptFiles", "true")
        .schema(streaming_logs_schema)
        .json(RAW_PATH)
        .where(F.col("timestamp").isNotNull())
        .withColumn("log_file", F.input_file_name())
    )

    streaming_logs_sdf.writeStream.format("delta").outputMode("append").option(
        "checkpointLocation", CHECK_POINT
    ).trigger(availableNow=True).option("overwriteSchema", "true").option(
        "mergeSchema", "true"
    ).toTable(
        BRONZE_TABLE
    ).awaitTermination()
    print('BRONZE UPDATED')

In [48]:
speed_test()
update_bronze_table()

24/04/15 21:52:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/15 21:52:14 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/04/15 21:52:14 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/15 21:52:14 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [12]:
sql(f """ SELECT * FROM {BRONZE_TABLE} ORDER BY timestamp DESC LIMIT 5 """ )

,timestamp,download,ping,isp,result,server,type,upload,log_file
0,2024-04-15 21:07:36,"(62536717, 702644896, None, {""iqm"":34.165,""low...","(4.410999774932861, 12.133000373840332, 5.0370...",Tigo Panama,"(f53576d3-0ef1-4ad7-8b32-d9d1563387a7, https:/...","(31830, speedtest.ufinet.com.pa, 8080, Ufinet,...",result,"(1855289, 6678176, None, {""iqm"":7.396,""low"":5....",file:///home/hadoop/deltalake/speed_test/raw_d...
1,2024-04-15 21:06:39,"(59707854, 402856768, None, {""iqm"":35.229,""low...","(4.589000225067139, 12.3100004196167, 11.20400...",Tigo Panama,"(9e00ef59-e13c-40c3-afc0-20345fe7edcb, https:/...","(28116, speed1.transoceanet.com, 8080, Trans O...",result,"(1852077, 7229864, None, {""iqm"":10.868,""low"":5...",file:///home/hadoop/deltalake/speed_test/raw_d...
2,2024-04-15 21:05:23,"(64217079, 918590928, None, {""iqm"":62.217,""low...","(3.1600000858306885, 12.675000190734863, 10.70...",Tigo Panama,"(262c8820-faed-4457-b4fe-382ccb1ffa7f, https:/...","(31830, speedtest.ufinet.com.pa, 8080, Ufinet,...",result,"(1835840, 6802704, None, {""iqm"":7.065,""low"":5....",file:///home/hadoop/deltalake/speed_test/raw_d...
3,2024-04-15 19:33:25,"(52752789, 701208480, None, {""iqm"":25.712,""low...","(2.9070000648498535, 11.720999717712402, 10.47...",Tigo Panama,"(a37a478c-5ec3-450d-9d4d-5e17ddfe5e1f, https:/...","(11318, speedtest.shadwell.com.pa, 8080, Shadw...",result,"(1849241, 7630960, None, {""iqm"":12.642,""low"":4...",file:///home/hadoop/deltalake/speed_test/raw_d...
4,2024-04-15 19:32:49,"(55521283, 511020920, None, {""iqm"":32.806,""low...","(4.864999771118164, 10.866000175476074, 6.9739...",Tigo Panama,"(f7693fad-e747-4d87-b444-3bfd403d2c36, https:/...","(40265, velocidad.pacificnetcom.com, 8080, Pac...",result,"(1859380, 7835128, None, {""iqm"":7.726,""low"":5....",file:///home/hadoop/deltalake/speed_test/raw_d...


### Silver layer (cleansed and conformed data)
In the Silver layer of the lakehouse, the data from the Bronze layer is matched, merged, conformed and cleansed ("just-enough") so that the Silver layer can provide an "Enterprise view" of all its key business entities, concepts and transactions. (e.g. master customers, stores, non-duplicated transactions and cross-reference tables).

The Silver layer brings the data from different sources into an Enterprise view and enables self-service analytics for ad-hoc reporting, advanced analytics and ML. It serves as a source for Departmental Analysts, Data Engineers and Data Scientists to further create projects and analysis to answer business problems via enterprise and departmental data projects in the Gold Layer.

In the lakehouse data engineering paradigm, typically the ELT methodology is followed vs. ETL - which means only minimal or "just-enough" transformations and data cleansing rules are applied while loading the Silver layer. Speed and agility to ingest and deliver the data in the data lake is prioritized, and a lot of project-specific complex transformations and business rules are applied while loading the data from the Silver to Gold layer. From a data modeling perspective, the Silver Layer has more 3rd-Normal Form like data models. Data Vault-like, write-performant data models can be used in this layer.

#### Change data feed
Change Data Feed (CDF) feature allows Delta tables to track row-level changes between versions of a Delta table. When enabled on a Delta table, the runtime records “change events” for all the data written into the table. This includes the row data along with metadata indicating whether the specified row was inserted, deleted, or updated.

You can read the change events in batch queries using DataFrame APIs (that is, df.read) and in streaming queries using DataFrame APIs (that is, df.readStream).

Use cases
Change Data Feed is not enabled by default. The following use cases should drive when you enable the change data feed.

Silver and Gold tables: Improve Delta performance by processing only row-level changes following initial MERGE, UPDATE, or DELETE operations to accelerate and simplify ETL and ELT operations.

Transmit changes: Send a change data feed to downstream systems such as Kafka or RDBMS that can use it to incrementally process in later stages of data pipelines.

Audit trail table: Capture the change data feed as a Delta table provides perpetual storage and efficient query capability to see all changes over time, including when deletes occur and what updates were made.


| Column name | Type | Values |  
|-------------|----------|------------------------|
| _change_type  | String  | insert, update_preimage , update_postimage, delete  | 
| _commit_version | Long  | The Delta log or table version containing the change. | 
| _commit_timestamp | Timestamp |The timestamp associated when the commit was created. |


### Use cases
Change Data Feed is not enabled by default. The following use cases should drive when you enable the change data feed.

Silver and Gold tables: Improve Delta performance by processing only row-level changes following initial MERGE, UPDATE, or DELETE operations to accelerate and simplify ETL and ELT operations.

Transmit changes: Send a change data feed to downstream systems such as Kafka or RDBMS that can use it to incrementally process in later stages of data pipelines.

Audit trail table: Capture the change data feed as a Delta table provides perpetual storage and efficient query capability to see all changes over time, including when deletes occur and what updates were 

### Enable change data feed

- New table: Set the table property delta.enableChangeDataFeed = true in the CREATE TABLE command:<br>```CREATE TABLE student (id INT, name STRING, age INT) TBLPROPERTIES (delta.enableChangeDataFeed = true)```
  
- Existing table: Set the table property delta.enableChangeDataFeed = true in the ALTER TABLE command.<br>```ALTER TABLE myDeltaTable SET TBLPROPERTIES (delta.enableChangeDataFeed = true)```

- All new tables:<br>
      ```set spark.databricks.delta.properties.defaults.enableChangeDataFeed = true```made.



In [61]:
def update_silver_sdf():
    bronze_delta_table = DeltaTable.forName(spark,BRONZE_TABLE)
    silver_delta_table = DeltaTable.forName(spark,SILVER_TABLE)
    def clean_bronze_table(sdf):
        sdf = (
        sdf
        .select(
            [
                F.col("result").getItem("id").alias("test_id"),
                F.col("timestamp"),
                F.to_date("timestamp").alias("date"),
                F.hour("timestamp").alias("hour"),
                "download",
                "upload",
                'isp',
               F.col('server').getItem('name').alias('server_name'),
                F.col('server').getItem('ip').alias('server_ip'),
                F.col('server').getItem('location').alias('server_location'),
                F.col('server').getItem('country').alias('server_country')
            ]
        )
        .withColumn(
            "part_of_the_day",
            (
                F.when((F.col("hour") >= 5) & (F.col("hour") < 12), "Morning")
                .when((F.col("hour") >= 12) & (F.col("hour") < 17), "Afternoon")
                .when((F.col("hour") >= 17) & (F.col("hour") < 21), "Evening")
                .otherwise("Night")
            ),
        )
        .withColumn("download_Mbytes", F.col("download").getItem("bytes") / 1000000)
        .withColumn("upload_Mbytes", F.col("upload").getItem("bytes") / 1000000)
        .drop('download')
        .drop('upload')
            )
        return sdf
        
    if silver_delta_table.toDF().limit(1).count() == 0:
        
        
        sdf = clean_bronze_table(spark.read.format('delta').table(BRONZE_TABLE))
        (sdf.write.format('delta').mode("append").option(
        "mergeSchema", "true").saveAsTable(SILVER_TABLE)
        )
        print('SILVER FIRST LOAD')
    else: 
        bronze_last_update = bronze_delta_table.history().where('operation != "CREATE TABLE"').select(F.max('timestamp').alias('bronze_last_update')).collect()[0][0]
        silver_last_update = silver_delta_table.history().where('operation != "CREATE TABLE"').select(F.max('timestamp').alias('bronze_last_update')).collect()[0][0]
        if bronze_last_update > silver_last_update:
            sdf = clean_bronze_table(spark.read.format('delta')
                .option("readChangeFeed", "true")\
                    .option("startingTimestamp", str(silver_last_update))
                  .table(BRONZE_TABLE))
            silver_delta_table.alias('sink').merge(sdf.alias('source'),'source.test_id = sink.test_id').whenNotMatchedInsertAll().execute()
            print('SILVER UPDATED')
        else:
            print('No updates')

In [62]:
speed_test()
update_bronze_table()
update_silver_sdf()

24/04/15 21:56:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/15 21:56:34 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/04/15 21:56:34 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/15 21:56:34 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


SILVER UPDATED


In [64]:
sql(f'DESCRIBE HISTORY {SILVER_TABLE}').operationMetrics.iloc[0]

{'numOutputRows': '1',
 'numTargetBytesAdded': '3931',
 'numTargetRowsInserted': '1',
 'numTargetFilesAdded': '1',
 'numTargetRowsMatchedDeleted': '0',
 'numTargetFilesRemoved': '0',
 'numTargetRowsMatchedUpdated': '0',
 'executionTimeMs': '524',
 'numTargetDeletionVectorsUpdated': '0',
 'numTargetRowsCopied': '0',
 'rewriteTimeMs': '523',
 'numTargetRowsUpdated': '0',
 'numTargetDeletionVectorsRemoved': '0',
 'numTargetRowsDeleted': '0',
 'scanTimeMs': '0',
 'numSourceRows': '1',
 'numTargetDeletionVectorsAdded': '0',
 'numTargetChangeFilesAdded': '0',
 'numTargetRowsNotMatchedBySourceUpdated': '0',
 'numTargetRowsNotMatchedBySourceDeleted': '0',
 'numTargetBytesRemoved': '0'}

In [ ]:
plot_bgcolor = "#2c292d"
# paper_bgcolor ="#211f22"
paper_bgcolor = "#1a1d21"
download_color = "#ab9df2"
upload_color = "#78dce8"
default_fontcolor = "white"

import dash
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from dash import dcc, html
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots


def line_chart_download_vs_upload(fig, df):

    x = df.log_timestamp

    fig.add_trace(
        go.Scatter(
            x=x, y=df.upload_Mbytes, name="Upload (Mbps)", line=dict(color=upload_color)
        ),
        row=1,
        col=2,
    )
    fig.update_yaxes(
        title_text="<b>Upload (Mbps)</b>",
        color=upload_color,
        rangemode="tozero",
        showgrid=False,
        row=1,
        col=2,
    )

    fig.add_trace(
        go.Scatter(
            x=x,
            y=df.download_Mbytes,
            name="Download (Mbps)",
            line=dict(color=download_color),
        ),
        row=2,
        col=2,
    )

    fig.update_yaxes(
        title_text="<b>Download (Mbps)</b>",
        color=download_color,
        rangemode="tozero",
        showgrid=False,
        row=2,
        col=2,
    )

    fig.update_xaxes(
        showgrid=False,
    )

    fig.update_layout(
        plot_bgcolor=plot_bgcolor,
        paper_bgcolor=paper_bgcolor,
        font=dict(color="white"),
        legend=dict(orientation="h", yanchor="bottom", y=-0.15, xanchor="right", x=1),
    )


def gauges_indicators(fig, value):

    def gauge_chart(value, steps, title, color):
        max_step = steps[-1][-1]
        title = f"{title} <span style='font-size:0.8em;color:gray'>MBps</span><br><span style='font-size:0.5em;color:gray'>Average</span>"
        gauge = go.Indicator(
            mode="gauge+number+delta",
            value=value,
            domain={"x": [0.25, 0.55], "y": [0.25, 0.55]},
            title={"text": title, "font": {"size": 25}, "align": "center"},
            delta={
                "reference": steps[-1][0],
                "font": {"size": 13},
                "increasing": {"color": color},
            },
            number={"font": {"size": 25}},
            gauge={
                "axis": {
                    "range": [None, max_step],
                    "tickwidth": 2,
                    "tickcolor": plot_bgcolor,
                },
                "bar": {"color": color},
                "bgcolor": "white",
                "borderwidth": 2,
                "bordercolor": "gray",
                "steps": [
                    {"range": steps[0], "color": "#ff6188"},
                    {"range": steps[1], "color": "#fc9867"},
                    {"range": steps[2], "color": "#a9dc76"},
                ],
            },
        )

        return gauge

    fig.add_trace(
        gauge_chart(
            value["upload_Mbytes"],
            steps=[[0, 10], [10, 15], [15, 20]],
            title=f"<span style='font-size:0.8em;color:{upload_color}'>Upload</span>",
            color=upload_color,
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        gauge_chart(
            value["download_Mbytes"],
            steps=[[0, 450], [450, 600], [600, 700]],
            title=f"<span style='font-size:0.8em;color:{download_color}'>Download</span>",
            color=download_color,
        ),
        row=2,
        col=1,
    )

    fig.update_layout(
        paper_bgcolor=paper_bgcolor,
        font={"color": "white", "family": "Arial"},
        showlegend=False,
    )
    fig.update_traces(number=dict(font=dict(size=28)), delta=dict(font=dict(size=25)))


def multiplot_speedtest(df):

    fig = make_subplots(
        rows=2,
        cols=2,
        specs=[[{"type": "domain"}, {}], [{"type": "domain"}, {}]],
        column_widths=[0.30, 0.70],
        row_heights=[0.25, 0.25],
        horizontal_spacing=0.15,
        vertical_spacing=0.15,
    )

    values = df.iloc[-3:].mean()
    gauges_indicators(fig, values)
    line_chart_download_vs_upload(fig, df)
    fig.update_layout(height=550, margin=dict(l=35, r=35, b=30, t=55))

    return fig


def register_Callback(app):
    @app.callback(
        Output("stream_line_chart", "figure"),
        [
            Input("interval-component", "n_intervals"),
        ],
    )
    def streamFig(intervals):
        df = (
            spark.read.table(BRONZE_TABLE)
            .where(
                F.col("log_timestamp")
                >= (F.current_timestamp() - F.expr("INTERVAL 60 minutes"))
            )
            .orderBy("log_timestamp", ascending=False)
            .limit(10)
            .toPandas()
            .sort_values("log_timestamp", ascending=True)
        )
        return multiplot_speedtest(df)


config = {"displaylogo": False, "scrollZoom": False, "displayModeBar": False}

updates = dcc.Interval(
    id="interval-component", interval=10000, n_intervals=0  # in milliseconds
)


navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(
                            html.Img(
                                src="https://www.pinclipart.com/picdir/big/491-4917274_panama-flag-png-palestine-flag-vector-clipart.png",
                                height="30px",
                            )
                        ),
                        dbc.Col(
                            dbc.NavbarBrand(
                                "Network Speed Test by Jose Quesada", className="ms-2"
                            )
                        ),
                    ],
                    align="center",
                    className="g-0",
                ),
                href="https://plotly.com",
                style={"textDecoration": "none"},
            ),
            dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
        ]
    ),
    color=paper_bgcolor,
    dark=True,
)


streaming_col = dbc.Col(dcc.Graph(id="stream_line_chart", config=config))

layout = dbc.Container(
    [
        navbar,
        dbc.Container(
            [
                updates,
                dcc.Store(id="last_32hrs"),
                dbc.Row(streaming_col),
            ],
            style={"background-color": paper_bgcolor, "color": default_fontcolor},
        ),
    ]
)

app = dash.Dash(
    external_stylesheets=[
        "https://cdn.jsdelivr.net/npm/bootstrap@4.3.1/dist/css/bootstrap.min.css"
    ],
)
# app.config.suppress_callback_exceptions = True
app.layout = layout
register_Callback(app)
app.run()

In [13]:
print(1)

1


24/04/14 20:32:46 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:32:46 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:32:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/04/14 20:32:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:32:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:33:07 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:33:07 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [ ]:
deltaTableLogs = DeltaTable.forName(spark, BRONZE_TABLE)

In [ ]:
windowedCounts = (
    words.withWatermark("timestamp", "10 minutes")
    .groupBy(window(words.timestamp, "10 minutes", "5 minutes"), words.word)
    .count()
)

In [ ]:
dfUpdates = deltaTablePeopleUpdates.toDF()

deltaTablePeople.alias("people").merge(
    dfUpdates.alias("updates"), "people.id = updates.id"
).whenMatchedUpdate(
    set={
        "id": "updates.id",
        "firstName": "updates.firstName",
        "middleName": "updates.middleName",
        "lastName": "updates.lastName",
        "gender": "updates.gender",
        "birthDate": "updates.birthDate",
        "ssn": "updates.ssn",
        "salary": "updates.salary",
    }
).whenNotMatchedInsert(
    values={
        "id": "updates.id",
        "firstName": "updates.firstName",
        "middleName": "updates.middleName",
        "lastName": "updates.lastName",
        "gender": "updates.gender",
        "birthDate": "updates.birthDate",
        "ssn": "updates.ssn",
        "salary": "updates.salary",
    }
).execute()